In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from datetime import datetime

In [3]:
full_co2 = pd.concat([pd.read_csv("./raw/FR_2021_hourly.csv"), pd.read_csv("./raw/FR_2022_hourly.csv")])

In [4]:
full_co2

,Datetime (UTC),Country,Zone Name,Zone Id,Carbon Intensity gCO₂eq/kWh (direct),Carbon Intensity gCO₂eq/kWh (LCA),Low Carbon Percentage,Renewable Percentage,Data Source,Data Estimated,Data Estimation Method
0,2021-01-01 00:00:00,France,France,FR,40.17,60.73,91.25,15.73,opendata.reseaux-energies.fr,False,NaN
1,2021-01-01 01:00:00,France,France,FR,39.88,60.44,91.50,16.52,opendata.reseaux-energies.fr,False,NaN
2,2021-01-01 02:00:00,France,France,FR,31.01,50.04,92.70,15.67,opendata.reseaux-energies.fr,False,NaN
3,2021-01-01 03:00:00,France,France,FR,26.88,45.11,93.29,15.49,opendata.reseaux-energies.fr,False,NaN
4,2021-01-01 04:00:00,France,France,FR,27.41,45.59,93.34,15.01,opendata.reseaux-energies.fr,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31 19:00:00,France,France,FR,21.31,39.08,95.06,44.69,opendata.reseaux-energies.fr,False,NaN
8756,2022-12-31 20:00:00,France,France,FR,20.90,38.70,95.04,44.33,opendata.reseaux-energies.fr,False,NaN
8757,2022-12-31 21:00:00,France,France,FR,21.85,39.99,94.82,44.51,opendata.reseaux-energies.fr,False,NaN
8758,2022-12-31 22:00:00,France,France,FR,21.27,39.31,94.94,44.18,opendata.reseaux-energies.fr,False,NaN


In [5]:
co2 = full_co2[['Datetime (UTC)', 'Carbon Intensity gCO₂eq/kWh (direct)']].copy()
co2.loc[:,'Datetime'] = pd.to_datetime(co2.loc[:,'Datetime (UTC)']) - pd.DateOffset(years=14)
co2.set_index('Datetime', inplace=True)
co2.loc[:, "Carbon Intensity gCO₂eq/kWmin (direct)"] = co2.loc[:, "Carbon Intensity gCO₂eq/kWh (direct)"] / 60
co2 = co2.loc[:, ['Carbon Intensity gCO₂eq/kWmin (direct)']]
co2 = co2.resample('min').interpolate(method='linear')

In [6]:
co2

,Carbon Intensity gCO₂eq/kWmin (direct)
Datetime,
2007-01-01 00:00:00,0.669500
2007-01-01 00:01:00,0.669419
2007-01-01 00:02:00,0.669339
2007-01-01 00:03:00,0.669258
2007-01-01 00:04:00,0.669178
...,...
2008-12-31 22:56:00,0.389656
2008-12-31 22:57:00,0.390283
2008-12-31 22:58:00,0.390911


In [7]:
co2_episodes = co2.groupby(pd.Grouper(freq='W'))

In [8]:
co2_episodes.describe()

Carbon Intensity gCO₂eq/kWmin (direct)                      \
                                            count      mean       std   
Datetime                                                                
2007-01-07                                10080.0  1.072211  0.353038   
2007-01-14                                10080.0  1.184804  0.200671   
2007-01-21                                10080.0  0.863216  0.258120   
2007-01-28                                10080.0  0.739209  0.267256   
2007-02-04                                10080.0  0.452082  0.158474   
...                                           ...       ...       ...   
2008-12-07                                10080.0  1.897596  0.150330   
2008-12-14                                10080.0  1.890821  0.197264   
2008-12-21                                10080.0  1.183549  0.402953   
2008-12-28                                10080.0  0.448523  0.097987   
2009-01-04                                 4261.0  0.371808  0.040595   

                                                              
                 min       25%       50%       75%       max  
Datetime                                                      
2007-01-07  0.413167  0.748064  1.175779  1.381604  1.638667  
2007-01-14  0.703061  1.035740  1.198079  1.341605  1.600833  
2007-01-21  0.415167  0.678653  0.881757  1.100052  1.361333  
2007-01-28  0.342428  0.467917  0.745786  0.962547  1.290333  
2007-02-04  0.291500  0.344774  0.370800  0.532414  0.876167  
...              ...       ...       ...       ...       ...  
2008-12-07  1.589500  1.781732  1.897185  2.011988  2.291500  
2008-12-14  1.446500  1.732788  1.901515  2.035399  2.373000  
2008-12-21  0.387678  0.868642  1.245474  1.538583  1.808333  
2008-12-28  0.258167  0.386953  0.442819  0.505504  0.675000  
2009-01-04  0.323667  0.345475  0.360056  0.379000  0.486000  

[105 rows x 8 columns]

In [9]:
co2_episodes = pd.concat([group.fillna(0) for name, group in co2_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [10]:
co2_episodes.describe()

Carbon Intensity gCO₂eq/kWmin (direct)                      \
                                            count      mean       std   
Datetime                                                                
2007-01-07                                10080.0  1.072211  0.353038   
2007-01-14                                10080.0  1.184804  0.200671   
2007-01-21                                10080.0  0.863216  0.258120   
2007-01-28                                10080.0  0.739209  0.267256   
2007-02-04                                10080.0  0.452082  0.158474   
...                                           ...       ...       ...   
2008-11-30                                10080.0  1.437826  0.393409   
2008-12-07                                10080.0  1.897596  0.150330   
2008-12-14                                10080.0  1.890821  0.197264   
2008-12-21                                10080.0  1.183549  0.402953   
2008-12-28                                10080.0  0.448523  0.097987   

                                                              
                 min       25%       50%       75%       max  
Datetime                                                      
2007-01-07  0.413167  0.748064  1.175779  1.381604  1.638667  
2007-01-14  0.703061  1.035740  1.198079  1.341605  1.600833  
2007-01-21  0.415167  0.678653  0.881757  1.100052  1.361333  
2007-01-28  0.342428  0.467917  0.745786  0.962547  1.290333  
2007-02-04  0.291500  0.344774  0.370800  0.532414  0.876167  
...              ...       ...       ...       ...       ...  
2008-11-30  0.623333  1.091316  1.443585  1.748733  2.226333  
2008-12-07  1.589500  1.781732  1.897185  2.011988  2.291500  
2008-12-14  1.446500  1.732788  1.901515  2.035399  2.373000  
2008-12-21  0.387678  0.868642  1.245474  1.538583  1.808333  
2008-12-28  0.258167  0.386953  0.442819  0.505504  0.675000  

[104 rows x 8 columns]

In [12]:
demand =  pd.HDFStore('./minutely/consumption.h5')
with pd.HDFStore('./minutely/intensity.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in co2_episodes:
        if name.date() == demand[f"eps_{i}"].index[-1].date():
            store[f'eps_{i}'] = group
            i += 1
file = pd.HDFStore('./minutely/intensity.h5')
print(len(file) == len(demand))
file.close()
demand.close()

True
